In [1]:
# 大約要跑 15 分鐘
from pyspark import SparkConf, SparkContext
import math, sys, glob, random
import sympy
import numpy as np
from nltk import ngrams, word_tokenize
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
from itertools import *
import csv

# from tqdm.notebook import trange, tqdm
plt.style.use('ggplot')
# conf=SparkConf().set("spark.default.parallelism", 4)
conf = SparkConf().setMaster("local[*]").setAppName("Final")
sc = SparkContext(conf=conf)

In [2]:
%%time
with open('HatePolitics_data_path.json', encoding='utf-8') as f:
    data_path=json.load(f)

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 23.8 ms


In [3]:
data_path[:10]

['./data/HatePolitics/HatePolitics/20181231/M.1546228463.A.CE7.json',
 './data/HatePolitics/HatePolitics/20181231/M.1546272003.A.8D2.json',
 './data/HatePolitics/HatePolitics/20100921/M.1285047249.A.2E4.json',
 './data/HatePolitics/HatePolitics/20181007/M.1538925219.A.801.json',
 './data/HatePolitics/HatePolitics/20181007/M.1538887508.A.6A1.json',
 './data/HatePolitics/HatePolitics/20181007/M.1538917911.A.4CD.json',
 './data/HatePolitics/HatePolitics/20061102/M.1162474517.A.E40.json',
 './data/HatePolitics/HatePolitics/20061102/M.1162452298.A.AF4.json',
 './data/HatePolitics/HatePolitics/20150814/M.1439529079.A.63D.json',
 './data/HatePolitics/HatePolitics/20150814/M.1439563129.A.4C3.json']

In [4]:
data = sc.parallelize(data_path)

In [5]:
%%time
# 從路徑轉換成 instance
def load_instance(x):
    with open(x) as f:
        d = json.load(f)
    return d

data = data.map(load_instance)
load_instance(data_path[0])

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.9 ms


{'article_id': 'M.1546228463.A.CE7',
 'article_title': '[討論] 韓國瑜:五大弊案，柯pㄧ個都沒除',
 'author': 'bob120400 (花滿樓)',
 'board': 'HatePolitics',
 'content': 'https://youtu.be/SBx80VZ5Yyw 影片約一分鐘處開始 韓國瑜認為，五大弊案柯文哲一個都沒除掉，因為柯文哲搞錯了自己首長的角色。 ',
 'date': 'Mon Dec 31 11:54:21 2018',
 'ip': '1.175.23.142',
 'message_conut': {'all': 290,
  'boo': 33,
  'count': 54,
  'neutral': 170,
  'push': 87},
 'messages': [{'push_content': '嗯嗯，韓加油，',
   'push_ipdatetime': '12/31 11:57',
   'push_tag': '→',
   'push_userid': 'lcw33242976'},
  {'push_content': '等著看賭場、摩天輪、迪士尼樂園那些能不能蓋得出來',
   'push_ipdatetime': '12/31 11:57',
   'push_tag': '→',
   'push_userid': 'lcw33242976'},
  {'push_content': '他的執政直接決定國民黨2020順風會不會翻船耶',
   'push_ipdatetime': '12/31 11:57',
   'push_tag': '→',
   'push_userid': 'Marchosias'},
  {'push_content': '看他覺得怎樣算處理吧...美河案打成那樣兩個公務人員',
   'push_ipdatetime': '12/31 11:58',
   'push_tag': '→',
   'push_userid': 'ayubabbit'},
  {'push_content': '出來頂罪...',
   'push_ipdatetime': '12/31 11:58',
   'push

In [6]:
len(data_path)

81650

In [7]:
data.take(1)

[{'article_id': 'M.1546228463.A.CE7',
  'article_title': '[討論] 韓國瑜:五大弊案，柯pㄧ個都沒除',
  'author': 'bob120400 (花滿樓)',
  'board': 'HatePolitics',
  'content': 'https://youtu.be/SBx80VZ5Yyw 影片約一分鐘處開始 韓國瑜認為，五大弊案柯文哲一個都沒除掉，因為柯文哲搞錯了自己首長的角色。 ',
  'date': 'Mon Dec 31 11:54:21 2018',
  'ip': '1.175.23.142',
  'message_conut': {'all': 290,
   'boo': 33,
   'count': 54,
   'neutral': 170,
   'push': 87},
  'messages': [{'push_content': '嗯嗯，韓加油，',
    'push_ipdatetime': '12/31 11:57',
    'push_tag': '→',
    'push_userid': 'lcw33242976'},
   {'push_content': '等著看賭場、摩天輪、迪士尼樂園那些能不能蓋得出來',
    'push_ipdatetime': '12/31 11:57',
    'push_tag': '→',
    'push_userid': 'lcw33242976'},
   {'push_content': '他的執政直接決定國民黨2020順風會不會翻船耶',
    'push_ipdatetime': '12/31 11:57',
    'push_tag': '→',
    'push_userid': 'Marchosias'},
   {'push_content': '看他覺得怎樣算處理吧...美河案打成那樣兩個公務人員',
    'push_ipdatetime': '12/31 11:58',
    'push_tag': '→',
    'push_userid': 'ayubabbit'},
   {'push_content': '出來頂罪...',
    'push_ipdate

In [8]:
%%time
TAG_SCORE = {'推':1, '噓':-1, '→':0}
def count_user_interaction(x, tag):
    used = ['author', 'messages']
    for i in used:
        if i not in x or x[i] == None:
            return [[]]
    main_user = x['author'].split(' ')[0]
    user_comments = defaultdict(lambda : 0)
    
    # 計算分數
    for i in x['messages']:
        
        uid = i['push_userid']
        uscore = tag[i['push_tag']]
        user_comments[uid] += uscore
    
    # 建立 graph
    pairs = []
    for i in user_comments:
        is_neg = user_comments[i]
        score = abs(user_comments[i])
        if score == 0:
            continue
        if is_neg:
            pair = [(i, main_user), score]
        else:
            pair = [(main_user, i), score]
        pairs.append(pair)
        
    return pairs

# count_user_interaction(load_instance(data_path[0]), TAG_SCORE)
data = data.flatMap(lambda x:count_user_interaction(x, TAG_SCORE)).filter(lambda x:x!=[])
data.take(5)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 43.6 ms


[[('Marchosias', 'bob120400'), 1],
 [('ayubabbit', 'bob120400'), 1],
 [('pingtong0602', 'bob120400'), 1],
 [('AndyZer', 'bob120400'), 1],
 [('william12tw', 'bob120400'), 1]]

In [9]:
%%time
data = data.groupByKey().mapValues(sum)
# data = data.repartition(1).groupByKey().mapValues(sum).repartition(1).cache()

data.take(5)



CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 6.91 s


[(('william12tw', 'bob120400'), 1),
 (('momo28', 'bob120400'), 1),
 (('PttObba', 'bob120400'), 25),
 (('ice80712', 'bob120400'), 1),
 (('smallkaka', 'bob120400'), 1)]

In [10]:
data.saveAsTextFile('ttt')

In [11]:
%%time
normalize_weight = data.map(lambda x: (x[0][0], x[1] )).groupByKey().mapValues(lambda x: (sum(x), len(x))).repartition(1)
normalize_weight.take(5)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 38.8 s


[('blackacre', (362, 326)),
 ('thedin', (5, 5)),
 ('tabris0122', (1, 1)),
 ('alubar', (14, 13)),
 ('chia0220', (230, 212))]

In [12]:
normalize_weight.saveAsTextFile('u2u_interactive_graph_norm_weight')

In [13]:
%%time
normalize_weight = normalize_weight.collect()
NWeight = {}
for i in normalize_weight:
    NWeight[i[0]] = i[1]
data_norm = data.map(lambda x: (x[0], x[1]/NWeight[x[0][0]][1]))
data_norm.take(5)

CPU times: user 3.72 s, sys: 80 ms, total: 3.8 s
Wall time: 7.67 s


[(('blackacre', 'yryangs'), 0.003067484662576687),
 (('thedin', 'KingKingCold'), 0.2),
 (('tabris0122', 'autokey'), 1.0),
 (('alubar', 'hugh509'), 0.07692307692307693),
 (('chia0220', 'Rilex'), 0.0047169811320754715)]

In [14]:
normalize_weight.saveAsTextFile('u2u_interactive_graph_norm')

AttributeError: 'list' object has no attribute 'saveAsTextFile'

In [ ]:
# %%time
# graph = data.collect()

# graph[0]


In [ ]:
# with open('graph.csv', 'w') as f:
#     json.dump(graph, f, indent=4, ensure_ascii=False)
# graph[0]

In [ ]:
############################################################

In [ ]:
# user to page
